In [ ]:
from earthchem.documentation import *
from earthchem.query import RESTClientQuery

In [ ]:
docs = get_documentation()

In [ ]:
q = RESTClientQuery(keyword='basalt', author='barnes')

In [ ]:
q.url

In [ ]:
q

In [ ]:
q?